In [ ]:
import numpy as np
from io import StringIO

from scripts import morphoGrid as morph
from scipy.spatial import cKDTree

import meshio

To quantify the impact of geomorphology on the evolution of terrestrial biota, we extract local physiographic resistance to species displacement 

# LEC & landscape parameters

We first start by reading the landscape parameters from the **badlands** run.

In [ ]:
step = 700
dx = 5000.
folder = 'lem-models/Sarr/h5'

In [ ]:
badland_topo = morph.morphoGrid(folder=folder, bbox = None, dx=dx)
badland_topo.loadHDF5(timestep=step)
badland_topo.getParams()
sealevel = badland_topo.sl
cells = badland_topo.cells-1
vertices = badland_topo.vertices
slope = badland_topo.grad

vertices[:,-1] -= sealevel

# Regular grid coordinates
xyi = np.dstack([badland_topo.x.flatten(), badland_topo.y.flatten()])[0]

# TIN grid coordinates
XY = vertices[:,:2] 
tin_tree = cKDTree(XY)

# On the uniform grid
distances, indices = tin_tree.query(xyi, k=3)

onIDs = np.where(distances[:, 0] == 0)[0]
inIDs = np.where(distances[:, 0] > 0)[0]

weights = np.ones(distances.shape)
weights[inIDs,:] = 1.0 / distances[inIDs,:] ** 2

denum = 1.0 / np.sum(weights, axis=1)

The landscape elevational connectivity has been calculated in the previous notebook. We will read the `VTK` file that was created:

In [ ]:
costmesh = 'vtk/Sarr700.cost.vtk'

# Get costs 
mesh = meshio.read(costmesh)

elev = mesh.point_data['Z']
lec = mesh.point_data['cost']
lec = 1.0-lec/lec.max()

ED = mesh.point_data['ED']
FA = mesh.point_data['FA']

## Additional geomorphic costs

The proposed cost surfaces are designed to represent long-term suitability of the landscape to movement of non-volant terrestrial species and combine the following three natural landscape features: 
 + (1) the normalized landscape elevational connectivity metric 
 + (2) the distance to main river systems and 
 + (3) the average local slope. 
 
 
We also consider that hydrological connectivity of aquatic habitats can either enhances the movement of terrestrial organisms (corridors) or hamper it (barriers to movement). We also assume that marine intrusion acts as barrier as well. 


To keep the analysis simple, costs related to each feature are equally weighted with categorical values ranging from 0 to 20. 

### Distance to river costs

In [ ]:
# set a limit on flow accumulation to only consider large rivers and 
# use kdtree to find closest points to these rivers on the mesh
logFA = np.log10(FA) / 13. # 13 is coming from logFA.max() at step 430 when river discharge is maximum
riverIDs = np.where(logFA>0.7)[0]

rtin_tree = cKDTree(XY[riverIDs,:]) 
dist, ids = rtin_tree.query(XY, k=1)
dist[elev<0] = 1.e18 # marine points set to large distance

# Define categorical distances cost based on user requirement
landIDs = np.where(elev>=0)[0]

# Barrier
river_barrier = np.zeros(len(dist))
river_barrier[elev<0] = 20.
river_barrier[landIDs] = 0.
river_barrier[dist<50000] = 1.
river_barrier[dist<35000] = 2.
river_barrier[dist<25000] = 3.
river_barrier[dist<20000] = 5.
river_barrier[dist<15000] = 7.
river_barrier[dist<10000] = 9.
river_barrier[dist<7000] = 11.
river_barrier[dist<5000] = 14.
river_barrier[dist<1000] = 20.

# Corridor
river_corridor = np.zeros(len(dist))
river_corridor[elev<0] = 20.
river_corridor[landIDs] = 20.
river_corridor[dist<50000] = 14.
river_corridor[dist<35000] = 11.
river_corridor[dist<25000] = 9.
river_corridor[dist<20000] = 7.
river_corridor[dist<15000] = 5.
river_corridor[dist<10000] = 3.
river_corridor[dist<7000] = 2.
river_corridor[dist<5000] = 1.
river_corridor[dist<1000] = 0.

### Elevation related costs

In [ ]:
# Define categorical distances cost based on user requirement
landIDs = np.where(elev>=0)[0]
cost_elev = np.zeros(len(dist))

cost_elev[elev<0] = 20.
cost_elev[landIDs] = 20.
cost_elev[elev<1000] = 15.
cost_elev[elev<250] = 5.
cost_elev[elev<100] = 0.

### Slope related costs

In [ ]:
# Define categorical slope cost based on user requirement
fslope = slope.flatten()

cost_slope = np.zeros(len(fslope))+20.
cost_slope[fslope<0.01] = 16
cost_slope[fslope<0.0075] = 12
cost_slope[fslope<0.005] = 10
cost_slope[fslope<0.0025] = 8
cost_slope[fslope<0.001] = 6
cost_slope[fslope<0.0005] = 2
cost_slope[fslope<0.0001] = 1

cost_slope = np.reshape(cost_slope, badland_topo.x.shape)

The resistance maps are then combined to produce regular cost surfaces grids (5 km side squared cells), ranked from high cost (>30) that are impermeable to species movement (*e.g.*, ocean, remote/close from/to river drainage network, steep local slope and low elevational connectivity) to low cost (<10) when permeable to movement (*e.g.*, close/far to rivers, low local slope and high elevational connectivity).

In [ ]:
# On the mesh
corridorCost = lec*20. + river_corridor + cost_elev
barrierCost = lec*20. + river_barrier + cost_elev

corridorCost_reg = np.sum(weights * corridorCost[indices], axis=1) * denum
barrierCost_reg = np.sum(weights * barrierCost[indices], axis=1) * denum
if len(onIDs) > 0:
    corridorCost_reg[onIDs] = corridorCost[indices[onIDs, 0]]
    barrierCost_reg[onIDs] = barrierCost[indices[onIDs, 0]]
    

corridorCost_reg = np.reshape(corridorCost_reg, badland_topo.x.shape)
corridorCost_reg += cost_slope

barrierCost_reg = np.reshape(barrierCost_reg, badland_topo.x.shape)
barrierCost_reg += cost_slope

corridorCost_reg = corridorCost_reg + 1
np.place(corridorCost_reg, badland_topo.z < sealevel, -9999)

barrierCost_reg = barrierCost_reg + 1
np.place(barrierCost_reg, badland_topo.z < sealevel, -9999)

We then store the cost surfaces as `VTK` files that can be visualised with Paraview.

In [ ]:
vtkfile = 'vtk/Sarr700.corridor.cost.vtk'
vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, 
                       point_data={"Z":elev, "ED":ED, 
                       "costLEC":lec*20, 'FA':logFA, 'costRiver':river_corridor,
                       "costElev":cost_elev,"all":corridorCost,
                      })
meshio.write(vtkfile, vis_mesh)

In [ ]:
vtkfile = 'vtk/Sarr700.barrier.cost.vtk'
vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, 
                       point_data={"Z":elev, "ED":ED, 
                       "costLEC":lec*20, 'FA':logFA, 'costRiver':river_barrier,
                       "costElev":cost_elev,"all":barrierCost,
                      })
meshio.write(vtkfile, vis_mesh)

## Input for circuitscape

We used circuit theory to evaluate the roles of geomorphology on past movements across the exposed Sunda Shelf over the last million years and extract preferential pathways over time. 

We chose [Circuitscape](https://circuitscape.org) to model multiple pathways. The software uses random walk and electric current running through a circuit. Electric current runs across our cost surfaces between predefined source points. We position these points across Sundaland (approximately 250 km apart) chosen along the outer margin (approx. 100 m above sea-level) of the maximum fully submerged shelf coastline.

In [ ]:
costMap1 = 'cost/Sarr_'+str(step)+'_corridor.asc'
costMap2 = 'cost/Sarr_'+str(step)+'_barrier.asc'


cost1 = np.flipud(corridorCost_reg.astype(dtype=int))
cost2 = np.flipud(barrierCost_reg.astype(dtype=int))

f1 = StringIO()
np.savetxt(f1, cost1, fmt='%.3f')
f1.seek(0)
fs1 = f1.read().replace('.000', '', -1)
f1.close()

f2 = StringIO()
np.savetxt(f2, cost2, fmt='%.3f')
f2.seek(0)
fs2 = f2.read().replace('.000', '', -1)
f2.close()

f1 = open(costMap1, 'w')
f1.write("ncols " + str(cost1.shape[1]) + "\n")
f1.write("nrows " + str(cost1.shape[0]) + "\n")
f1.write("xllcorner " + str(0) + "\n")
f1.write("yllcorner " + str(0) + "\n")
f1.write("cellsize " + str(5000) + "\n")
f1.write("NODATA_value " + str(-9999) + "\n")
f1.write(fs1)
f1.close()


f2 = open(costMap2, 'w')
f2.write("ncols " + str(cost1.shape[1]) + "\n")
f2.write("nrows " + str(cost1.shape[0]) + "\n")
f2.write("xllcorner " + str(0) + "\n")
f2.write("yllcorner " + str(0) + "\n")
f2.write("cellsize " + str(5000) + "\n")
f2.write("NODATA_value " + str(-9999) + "\n")
f2.write(fs1)
f2.close()

These maps are then used in **Circuitscape** using the settings presented in the figure below:

<div align="center">
    <img width=1000 src="img/Circuitscape.png?raw=true" alt="circuitscape setting" title="circuitscape setting"</img>
</div>